<a href="https://colab.research.google.com/github/Azmaininqiad/Robotic_1/blob/main/Robotic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai
!pip install --upgrade google-cloud-aiplatform
!pip install -q -U google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.2 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.74.0
    Uninstalling google-cloud-aiplatform-1.74.0:
      Successfully uninstalled google-cloud-aiplatform-1.74.0


Text Extraction


In [ ]:
from google.colab import userdata
#userdata.get('GOOGLE_API_KEY')

import google.generativeai as genai
from IPython.display import Markdown
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

# Upload the file and print a confirmation.
sample_file = genai.upload_file(path="/content/udvash.jpg",
                            display_name="Jetpack drawing")

print(f"Uploaded file '{sample_file.display_name}' as: {sample_file.uri}")
file = genai.get_file(name=sample_file.name)
print(f"Retrieved file '{file.display_name}' as: {sample_file.uri}")

# Choose a Gemini API model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Prompt the model with text and the previously uploaded image.
response = model.generate_content([sample_file, "Extract the text and describe the image from this picture"])

Markdown(">" + response.text)


Uploaded file 'Jetpack drawing' as: https://generativelanguage.googleapis.com/v1beta/files/ydb2zb0z4qfo
Retrieved file 'Jetpack drawing' as: https://generativelanguage.googleapis.com/v1beta/files/ydb2zb0z4qfo


>The image shows a handwritten calculation. It appears to be a physics problem, likely involving current (I), area (A), velocity (v), and possibly electron density (n) and charge (e).

Here's a breakdown of the handwritten text:

* **Line 1:** I = nAv<sub>c</sub>  (This is the standard equation for current density, where v<sub>c</sub> represents the drift velocity of charge carriers.)
* **Line 2:** v<sub>c</sub> = I / nAe  (This equation rearranges the first to solve for drift velocity.)
* **Line 3:** = (8x10<sup>15</sup> * 5x10<sup>-6</sup> * 1.6x10<sup>-19</sup>) (This line substitutes values into the equation. It seems 8x10<sup>15</sup> is the current (I), 5x10<sup>-6</sup> is the area (A), and 1.6x10<sup>-19</sup> is the elementary charge (e). The value for 'n' (number density of charge carriers) seems to be missing or perhaps is assumed to be 1.)
* **Line 4:** = 3.84x10<sup>-11</sup> (This is the calculated result for v<sub>c</sub>, the drift velocity.)


The writing is on lined paper within a bordered box, suggesting it might be part of an exam or homework assignment.  The number "11" at the top left likely indicates the question number.

In [ ]:
response_text = response.text

In [ ]:

! pip install -q --upgrade google-generativeai langchain-google-genai chromadb pypdf
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.5 MB/s eta 0:00:

RAG Application


In [ ]:
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA



warnings.filterwarnings("ignore")
# restart python kernal if issues with langchain import.

from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash",google_api_key=GOOGLE_API_KEY,system_instruction="You are an examiner. You will give marks according to the marks distribution according to the passage. if it doesnt match you will give 0. For writing formulas or other things, give the marks according to marks distribution. Explain reason for every marks. you have to write the total marks boldly at last",
                             temperature=0.5,convert_system_message_to_human=True)

pdf_loader = PyPDFLoader("/content/2.pdf")
pages = pdf_loader.load_and_split()
#print(pages[3].page_content)

from langchain_google_genai import GoogleGenerativeAIEmbeddings

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
context = "\n\n".join(str(p.page_content) for p in pages)
texts = text_splitter.split_text(context)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY)
vector_index = Chroma.from_texts(texts, embeddings).as_retriever(search_kwargs={"k":5})
qa_chain = RetrievalQA.from_chain_type(
    model,

    retriever=vector_index,
    return_source_documents=True

)

#question = "explain properties of space and time?"
#response = model.generate_content([sample_file, "Extract the text and describe the image from this picture"])
result = qa_chain({"query": response_text})
result["result"]

'The provided handwritten calculation uses an incorrect formula and values.  The correct formula, as shown in the sample answer, is v = j/(ne), where j is current density, n is the number of free electrons, and e is the elementary charge (1.6 x 10⁻¹⁹ C).  The handwritten calculation uses a formula involving current (I) and cross-sectional area (A), which is not directly given in the problem. Therefore, the drift velocity calculated in the handwritten image is incorrect.  Based on the provided marking scheme, a score of 0 would be given.\n'

In [ ]:
PyPDFLoader("/content/text.pdf")
pdf_loader.load_and_split()
RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [ ]:
# Load the PDF
pdf_loader = PyPDFLoader("/content/1.pdf")
pages = pdf_loader.load_and_split()

# Display the content of each page
for i, page in enumerate(pages):
    print(f"Page {i + 1}:\n{page.page_content}\n")


Page 1:
Question: At 20°C, the current density of the conductor is 3 x 10⁴ Am⁻² 
and the number of free electrons is 8 x 10²⁵ m⁻³. What is the drift 
velocity?
The sample answer provides the following solution:
• Formula: I = nAve, which simpliﬁes to j = nve. Solving for drift 
velocity (v) gives v = j / (ne)
• Calculation: v = (3 x 10⁴) / (8 x 10²⁵ x 1.6 x 10⁻¹⁹)
• Result: v = 2.34 x 10⁻³ ms⁻¹
The marks distribution for the question is:
• 3 marks for writing the correct formula (j = nve).
• 4 marks for correctly substituting the values into the formula.
• 3 marks for arriving at the correct drift velocity and including the 
unit (ms⁻¹).
The note clariﬁes that 1 mark will be deducted if the unit is missing or 
incorrect. It also states that directly inputting the values without writing 
the formula is acceptable.



In [ ]:
print(pages[0].page_content)  # First page content
print(pages[-1].page_content)  # Last page content


Question: At 20°C, the current density of the 
conductor is 3 x 10⁴ Am⁻² 
and the number of free electrons is 8 x 10²⁵ m⁻³. 
What is the drift 
velocity?
The sample answer provides the following solution:
• Formula: I = nAve, which simpliﬁes to j = nve. 
Solving for drift 
velocity (v) gives v = j / (ne)
• Calculation: v = (3 x 10⁴) / (8 x 10²⁵ x 1.6 x 
10⁻¹⁹)
• Result: v = 2.34 x 10⁻³ ms⁻¹
The marks distribution for the question is:
• 3 marks for writing the correct formula (j = nve).
• 4 marks for correctly substituting the values into 
the formula.
• 3 marks for arriving at the correct drift velocity 
and including the 
unit (ms⁻¹).
The note clariﬁes that 1 mark will be deducted if the 
unit is missing or 
incorrect. It also states that directly inputting the 
values without writing 
the formula is acceptable.
If the values and systems are incorrect then you will 
put 0 for the answer. Also it will give marks 
according to marks distribution if the formula or 
processing is valid.
Q

In [ ]:
from google.colab import userdata
#userdata.get('GOOGLE_API_KEY')

import google.generativeai as genai
from IPython.display import Markdown
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

# Upload the file and print a confirmation.
sample_file = genai.upload_file(path="/content/1.png",
                            display_name="Jetpack drawing")

print(f"Uploaded file '{sample_file.display_name}' as: {sample_file.uri}")
file = genai.get_file(name=sample_file.name)
print(f"Retrieved file '{file.display_name}' as: {sample_file.uri}")

# Choose a Gemini API model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Prompt the model with text and the previously uploaded image.
response = model.generate_content([sample_file, "Extract the text and describe the image from this picture"])

Markdown(">" + response.text)

Uploaded file 'Jetpack drawing' as: https://generativelanguage.googleapis.com/v1beta/files/3rs06lantlpv
Retrieved file 'Jetpack drawing' as: https://generativelanguage.googleapis.com/v1beta/files/3rs06lantlpv


>The image shows a physics question about calculating drift velocity. The question states:

**Question:** At 20°C, the current density of the conductor is 3 x 10⁴ Am⁻² and the number of free electrons is 8 x 10²⁵ m⁻³. What is the drift velocity?

The sample answer provides the following solution:

* **Formula:** I = nAve, which simplifies to j = nve.  Solving for drift velocity (v) gives v = j / (ne)
* **Calculation:** v = (3 x 10⁴) / (8 x 10²⁵ x 1.6 x 10⁻¹⁹)
* **Result:** v = 2.34 x 10⁻³ ms⁻¹

The marks distribution for the question is:

* 3 marks for writing the correct formula (j = nve).
* 4 marks for correctly substituting the values into the formula.
* 3 marks for arriving at the correct drift velocity and including the unit (ms⁻¹).

The note clarifies that 1 mark will be deducted if the unit is missing or incorrect. It also states that directly inputting the values without writing the formula is acceptable.

In [ ]:
!mkdir data
# Install required packages
!pip install -q transformers einops accelerate bitsandbytes langchain
!pip install pypdf sentence_transformers llama-index==0.9.39 langchain-community

In [ ]:
prompt="passage1 = [301 - 0.982, 2, 5.97,y.Daughter age = 4oy.Mother age 35.974.]passage2 = 02 marks for writing the formula t_{0} = t * sqrt(1 - (v ^ 2)/(e ^ (2pi))).02 marks for putting the values.02 marks for determining t_{0} = 5.97y.02 marks for determining age of daughter 40 years.02 marks for determining age of women 35.97 years.[Note: 01 mark will be deducted if unit is not written or wrong unit is written. Direct input of values without writing the formula is also acceptable.]"


In [ ]:
model=genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  system_instruction="You are an examiner. Based on the marks distribution provided in passage2, extract the total marks and the reason behind the marks for passage1.")

response = model.generate_content(prompt)
print(response.text)

Total marks for passage1: 12 marks

Reasoning:

Based on passage2's marking scheme,  passage1 would be marked as follows:

* **02 marks:**  While the formula isn't explicitly written, the presence of `t_{0} = t * sqrt(1 - (v ^ 2)/(e ^ (2pi)))` elements (within the passage1 string)  suggests the formula was used implicitly or at least considered.  The examiner might give credit for implied use.

* **02 marks:** Values (presumably  't', 'v', and 'e' ) seem to be present within passage1's jumbled data,  indicating the values were used.

* **02 marks:**  The result `t_{0} = 5.97y` is present in passage1.

* **02 marks:** The daughter's age (40 years) is explicitly stated in passage1.

* **02 marks:** The mother's age (35.97 years) is explicitly stated in passage1.

* **-2 marks (potential deduction):** Units may be missing or improperly formatted in the first part of passage1 (`301 - 0.982, 2`).  The marking scheme indicates a 1-mark deduction per missing/wrong unit, and the jumbled format